Modified from https://lectures.quantecon.org/jl/optgrowth.html

In [72]:
using QuantEcon, Optim, Distributions
using Optim,  Distributions, QuadGK, NamedTuples

## Some more on Numerical Integration

In [31]:
dist = Normal(0.1,2)
@show(minimum(dist),maximum(dist)) #The support
mean(dist) ≈ quadgk(x-> x * pdf(dist,x) , minimum(dist),maximum(dist))[1] #Numerical integraiton example

minimum(dist) = -Inf
maximum(dist) = Inf


true

In [42]:
#This sort of general utility could be added to the Distributions library, for example.  This works for any continous valued
function expectation(f, dist::D) where {D<:ContinuousUnivariateDistribution}
    quadgk( x-> f(x) * pdf(dist,x) , minimum(dist),maximum(dist))[1] #Uses Gauss-Kronod adaptive quadrature
end

#For a disscretely valued expectation, could just do the sums.  SHould check it is bounded
function expectation(f, dist::D) where {D<:DiscreteUnivariateDistribution}
    @assert hasfinitesupport(dist) #Otherwise might need to modify to use a truncated pdf.    
    dot(pdf.(dist, support(dist)),f.(support(dist))) #Returns the product of the pmf and the function
end



expectation (generic function with 4 methods)

In [43]:
#Test convenience operator
dist = Normal(0.1,2)
E(f) = expectation(f, dist) #Convenience

E(x -> x^2) - E(x -> x)^2 #i.e. calculate the variance

4.000000000000002

In [44]:
# Test the discrete distribution
dist = Binomial(10, 0.5)
E(f) = expectation(f, dist)
E(n -> n^2) - E2(n -> n)^2 #i.e. calculate the variance


2.5

## Solving the Stationary Problem

In [68]:
α = 0.4
β = 0.96
μ = 0
s = 0.1

c1 = log(1 - α * β) / (1 - β)
c2 = (μ + α * log(α * β)) / (1 - α)
c3 = 1 / (1 - β)
c4 = 1 / (1 - α * β)

# Utility 
u(c) = log(c)

u_prime(c) = 1 / c

# Deterministic part of production function
f(k) = k^α

f_prime(k) = α * k^(α - 1)

# True optimal policy
c_star(y) = (1 - α * β) * y

# True value function
v_star(y) = c1 + c2 * (c3 - c4) + c4 * log(y)

v_star (generic function with 1 method)

In [130]:
function T(w_grid, params)
    grid, β, u, f, shockdist = params.grid, params.β, params.u, params.f, params.shockdist #unpack
    
    w = LinInterp(grid, w_grid) #linear interpolation of w_grid
    E(f) = expectation(f, shockdist) #convenience operator

    Tw = [-optimize(c-> - u(c) - β * E(ζ -> w.(f.(y - c) .* exp(ζ))), 1e-10, y).minimum for y in grid]    
end

T (generic function with 1 method)

In [124]:
grid_max = 4         # Largest grid point
grid_size = 200      # Number of grid points


grid_y = collect(linspace(1e-5, grid_max, grid_size))


200-element Array{Float64,1}:
 1.0e-5   
 0.0201105
 0.0402109
 0.0603114
 0.0804118
 0.100512 
 0.120613 
 0.140713 
 0.160814 
 0.180914 
 0.201015 
 0.221115 
 0.241215 
 ⋮        
 3.7789   
 3.799    
 3.8191   
 3.8392   
 3.8593   
 3.8794   
 3.8995   
 3.9196   
 3.9397   
 3.9598   
 3.9799   
 4.0      

200-element Array{Float64,1}:
 -51.4925
 -33.3716
 -32.2476
 -31.5881
 -31.1217
 -30.7594
 -30.4633
 -30.213 
 -29.996 
 -29.8046
 -29.6336
 -29.479 
 -29.3376
   ⋮     
 -24.8706
 -24.862 
 -24.8534
 -24.8449
 -24.8364
 -24.828 
 -24.8196
 -24.8113
 -24.803 
 -24.7947
 -24.7865
 -24.7783

In [133]:
#Plotting the results
using Plots
gr()
plot(grid_y, w, lw=2, alpha=0.6, label="Tv^*")

params = @NT(grid=grid_y, β = β, u = log, f = f, shockdist = Normal(μ,s^2))
w = T(v_star.(grid_y), params) #Shock distribution
plot!(grid_y, v_star.(grid_y), lw=2, alpha=0.6, label="v^")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 -50 
 
 
 -45 
 
 
 -40 
 
 
 -35 
 
 
 -30 
 
 
 -25 
 
<polyline clip-path="url(#clip5602)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 36.1567,375.869 38.8912,128.92 41.6256,113.602 44.3601,104.614 47.0946,98.2584 49.829,93.321 52.5635,89.2857 55.298,85.8744 58.0324,82.918 60.7669,80.3092 
 63.5014,77.9785 66.2358,75.8723 68.9703,73.945 71.7047,72.1739 74.4392,70.5358 77.1737,69.0073 79.9081,67.5812 82.6426,66.2382 85.3771,64.975 88.1115,63.7775 
 90.846,62.6438 93.5805,61.5632 96.3149,60.535 99.0494,59.5506 101.784,58.6096 104.518,57.7062 107.253,56.8382 109.987,56.0038 112.722,55.1985 115.456,54.4226 
 118.191,53.6725 120.925,52.9467 123.66,52.2447 126.394,51.5636 129.128,50.903 131.863,50.2621 134.597,49.6385 137.332,49.0323 140.066,48.4426 142.801,47.8677 
 145.535,47.3074 148.27,46.7614 151.004,46.2282 153.739,45.7074 156.473,45.1989 159.208,44.7018 161.942,44.2154 164.677,43.7396 167.411,43.2739 170.145,42.8177 
 172.88,42.3706 175.614,41.9326 178.349,41.503 181.083,41.0815 183.818,40.6679 186.552,40.2621 189.287,39.8635 192.021,39.4718 194.756,39.087 197.49,38.7089 
 200.225,38.337 202.959,37.9713 205.693,37.6115 208.428,37.2576 211.162,36.9092 213.897,36.5661 216.631,36.2283 219.366,35.8956 222.1,35.5679 224.835,35.2449 
 227.569,34.9265 230.304,34.6127 233.038,34.3033 235.773,33.9982 238.507,33.6971 241.241,33.4001 243.976,33.1071 246.71,32.818 249.445,32.5325 252.179,32.2506 
 254.914,31.9723 257.648,31.6975 260.383,31.4261 263.117,31.1579 265.852,30.8929 268.586,30.6311 271.321,30.3723 274.055,30.1166 276.79,29.8637 279.524,29.6137 
 282.258,29.3665 284.993,29.1221 287.727,28.8803 290.462,28.6411 293.196,28.4045 295.931,28.1703 298.665,27.9387 301.4,27.7094 304.134,27.4825 306.869,27.2579 
 309.603,27.0355 312.338,26.8154 315.072,26.5974 317.806,26.3816 320.541,26.1678 323.275,25.9561 326.01,25.7464 328.744,25.5387 331.479,25.3329 334.213,25.129 
 336.948,24.9269 339.682,24.7267 342.417,24.5283 345.151,24.3316 347.886,24.1367 350.62,23.9435 353.354,23.752 356.089,23.562 358.823,23.3738 361.558,23.1871 
 364.292,23.0019 367.027,22.8183 369.761,22.6362 372.496,22.4556 375.23,22.2765 377.965,22.0988 380.699,21.9225 383.434,21.7476 386.168,21.5741 388.903,21.4019 
 391.637,21.2311 394.371,21.0616 397.106,20.8933 399.84,20.7263 402.575,20.5606 405.309,20.3961 408.044,20.2329 410.778,20.0708 413.513,19.9099 416.247,19.7501 
 418.982,19.5916 421.716,19.4341 424.451,19.2777 427.185,19.1225 429.919,18.9683 432.654,18.8152 435.388,18.6631 438.123,18.5121 440.857,18.3621 443.592,18.2132 
 446.326,18.0652 449.061,17.9182 451.795,17.7721 454.53,17.6271 457.264,17.4829 459.999,17.3398 462.733,17.1975 465.467,17.0561 468.202,16.9156 470.936,16.7761 
 473.671,16.6374 476.405,16.4995 479.14,16.3625 481.874,16.2264 484.609,16.0911 487.343,15.9566 490.078,15.8229 492.812,15.69 495.547,15.5579 498.281,15.4266 
 501.016,15.2961 503.75,15.1664 506.484,15.0374 509.219,14.9091 511.953,14.7816 514.688,14.6548 517.422,14.5288 520.157,14.4034 522.891,14.2788 525.626,14.1548 
 528.36,14.0316 531.095,13.909 533.829,13.7871 536.564,13.6659 539.298,13.5453 542.032,13.4254 544.767,13.3062 547.501,13.1875 550.236,13.0695 552.97,12.9522 
 555.705,12.8354 558.439,12.7193 561.174,12.6038 563.908,12.4888 566.643,12.3745 569.377,12.2608 572.112,12.1476 574.846,12.035 577.58,11.923 580.315,11.8115 
 
 "/>
<polyline clip-path="url(#clip5602)" style="stroke:#e26f46; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 36.1567,297.182 38.8912,128.905 41.6256,113.576 44.3601,104.607 47.0946,98.2437 49.829,93.3076 52.5635,89.2744 55.298,85.8644 58.0324,82.9105 60.7669,80.3049 
 63.5014,77.9741 66.2358,75.8656 68.9703,73.9408 71.7047,72.17 74.4392,70.5306 77.1737,69.0043 79.9081,67.5766 82.6426,66.2354 85.3771,64.9709 88.1115,63.7748 
 90.846,62.6401 93.5805,61.5607 

In [142]:
params = @NT(grid=grid_y, β = β, u = log, f = f, shockdist = Normal(μ,s^2))
w = 5 * log.(grid_y)  # An initial condition -- fairly arbitrary
n = 10

lb = "initial condition"
plot(grid_y, w)
w2 = T(w, params)
plot!(grid_y, w2)
w3 = T(w2, params)
plot!(grid_y, w3)
w4 = T(w3, params)
plot!(grid_y, w4)

#for i in 1:n
#    w = T(w, params)

#    plot!(grid_y, w)
#end
#display()

#lb = "true value function"
#plot(grid_y, v_star.(grid_y), "k-", lw=2, alpha=0.8, label=lb)

#show()

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 -50 
 
 
 -40 
 
 
 -30 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
<polyline clip-path="url(#clip6602)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 36.1567,375.869 38.8912,161.191 41.6256,141.635 44.3601,130.194 47.0946,122.076 49.829,115.779 52.5635,110.634 55.298,106.283 58.0324,102.515 60.7669,99.1909 
 63.5014,96.2175 66.2358,93.5276 68.9703,91.072 71.7047,88.813 74.4392,86.7215 77.1737,84.7744 79.9081,82.953 82.6426,81.242 85.3771,79.6288 88.1115,78.1029 
 90.846,76.6553 93.5805,75.2783 96.3149,73.9654 99.0494,72.7109 101.784,71.5097 104.518,70.3576 107.253,69.2507 109.987,68.1856 112.722,67.1592 115.456,66.1688 
 118.191,65.212 120.925,64.2866 123.66,63.3906 126.394,62.5221 129.128,61.6796 131.863,60.8614 134.597,60.0664 137.332,59.2931 140.066,58.5405 142.801,57.8074 
 145.535,57.0928 148.27,56.3959 151.004,55.7158 153.739,55.0517 156.473,54.4029 159.208,53.7686 161.942,53.1483 164.677,52.5414 167.411,51.9472 170.145,51.3652 
 172.88,50.795 175.614,50.2362 178.349,49.6881 181.083,49.1505 183.818,48.623 186.552,48.1051 189.287,47.5966 192.021,47.097 194.756,46.6062 197.49,46.1237 
 200.225,45.6494 202.959,45.1829 205.693,44.724 208.428,44.2724 211.162,43.8279 213.897,43.3903 216.631,42.9595 219.366,42.535 222.1,42.1169 224.835,41.7049 
 227.569,41.2988 230.304,40.8985 233.038,40.5037 235.773,40.1144 238.507,39.7304 241.241,39.3516 243.976,38.9778 246.71,38.6089 249.445,38.2447 252.179,37.8851 
 254.914,37.5301 257.648,37.1795 260.383,36.8332 263.117,36.4911 265.852,36.1531 268.586,35.8191 271.321,35.489 274.055,35.1627 276.79,34.8402 279.524,34.5213 
 282.258,34.2059 284.993,33.8941 287.727,33.5856 290.462,33.2805 293.196,32.9787 295.931,32.68 298.665,32.3845 301.4,32.092 304.134,31.8025 306.869,31.516 
 309.603,31.2323 312.338,30.9515 315.072,30.6734 317.806,30.3981 320.541,30.1254 323.275,29.8553 326.01,29.5878 328.744,29.3228 331.479,29.0603 334.213,28.8001 
 336.948,28.5424 339.682,28.287 342.417,28.0338 345.151,27.783 347.886,27.5343 350.62,27.2878 353.354,27.0435 356.089,26.8012 358.823,26.561 361.558,26.3228 
 364.292,26.0867 367.027,25.8524 369.761,25.6201 372.496,25.3898 375.23,25.1612 377.965,24.9345 380.699,24.7096 383.434,24.4865 386.168,24.2652 388.903,24.0455 
 391.637,23.8276 394.371,23.6113 397.106,23.3967 399.84,23.1837 402.575,22.9723 405.309,22.7625 408.044,22.5542 410.778,22.3474 413.513,22.1421 416.247,21.9384 
 418.982,21.736 421.716,21.5352 424.451,21.3357 427.185,21.1377 429.919,20.941 432.654,20.7457 435.388,20.5517 438.123,20.359 440.857,20.1677 443.592,19.9776 
 446.326,19.7888 449.061,19.6013 451.795,19.415 454.53,19.23 457.264,19.0461 459.999,18.8634 462.733,18.6819 465.467,18.5016 468.202,18.3224 470.936,18.1443 
 473.671,17.9674 476.405,17.7915 479.14,17.6168 481.874,17.4431 484.609,17.2705 487.343,17.0989 490.078,16.9284 492.812,16.7589 495.547,16.5904 498.281,16.4229 
 501.016,16.2564 503.75,16.0908 506.484,15.9263 509.219,15.7626 511.953,15.6 514.688,15.4382 517.422,15.2774 520.157,15.1175 522.891,14.9585 525.626,14.8004 
 528.36,14.6432 531.095,14.4868 533.829,14.3313 536.564,14.1767 539.298,14.0228 542.032,13.8699 544.767,13.7177 547.501,13.5664 550.236,13.4159 552.97,13.2661 
 555.705,13.1172 558.439,12.9691 561.174,12.8217 563.908,12.6751 566.643,12.5292 569.377,12.3841 572.112,12.2397 574.846,12.0961 577.58,11.9532 580.315,11.811 
 
 "/>
<polyline clip-path="url(#clip6602)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 36.1567,342.495 38.8912,125.942 41.6256,114.519 44.3601,107.83 47.0946,103.086 49.829,99.4046 52.5635,96.397 55.298,93.8571 58.0324,91.6563 60.7669,89.7144 
 63.5014,87.9771 66.2358,86.4057 68.9703,84.9719 71.7047,83.6533 74.4392,82.4313 77.1737,81.2932 79.9081,80.23 82.6426,79.2308 85.3771,78.2879 88.1115,77.3973 
 90.846,76.5515 93.5805,75.7472 96.3149,74.9806 9

In [61]:
function solve_optgrowth(initial_w; params,
                         tol::AbstractFloat=1e-6,
                         max_iter::Integer=500)

    w = initial_w  # Set initial condition
    error = tol + 1
    i = 0

    # == Create storage array for bellman_operator. Reduces  memory
    # allocation and speeds code up == #
    Tw = similar(grid_y)

    # Iterate to find solution

    while (error > tol) && (i < max_iter)
        w_new = T(w,params)
        error = maximum(abs, w_new - w)
        w = w_new
        i += 1
    end

    return w
end

solve_optgrowth (generic function with 1 method)

In [ ]:
initial_w = 5 * log.(grid_y)
v_star_approx = solve_optgrowth(initial_w)

fig, ax = subplots(figsize=(9, 5))
ax[:set_ylim](-35, -24)
ax[:plot](grid_y, v_star_approx, lw=2, alpha=0.6, label="approximate value function")
ax[:plot](grid_y, v_star.(grid_y), lw=2, alpha=0.6, label="true value function")
ax[:legend](loc="lower right")
show()

In [ ]:
import QuantEcon: compute_fixed_point

Tw = similar(grid_y)
initial_w = 5 * log.(grid_y)

bellman_operator(w) = bellman_operator(w,
                                       grid_y,
                                       β,
                                       log,
                                       k -> k^α,
                                       shocks)

v_star_approx = compute_fixed_point(bellman_operator,
                                    initial_w,
                                    max_iter=500,
                                    verbose=2,
                                    print_skip=10,
                                    err_tol=1e-5)

```none
Compute iterate 10 with error 0.709153897728406
Compute iterate 20 with error 0.47095844432889145
Compute iterate 30 with error 0.3131085083453158
Compute iterate 40 with error 0.20816475495214704
Compute iterate 50 with error 0.13839472275577336
Compute iterate 60 with error 0.09200932833702069
Compute iterate 70 with error 0.061170804294146564
Compute iterate 80 with error 0.04066834706496181
Compute iterate 90 with error 0.027037644380378367
Compute iterate 100 with error 0.01797550838462314
Compute iterate 110 with error 0.011950704623018282
Compute iterate 120 with error 0.00794521845620011
Compute iterate 130 with error 0.005282240529794535
Compute iterate 140 with error 0.003511805895449527
Compute iterate 150 with error 0.0023347631704915273
Compute iterate 160 with error 0.001552226753144481
Compute iterate 170 with error 0.0010319710041208907
Compute iterate 180 with error 0.0006860880028760619
Compute iterate 190 with error 0.0004561336956570017
Compute iterate 200 with error 0.00030325256715002524
Compute iterate 210 with error 0.00020161220411551994
Compute iterate 220 with error 0.0001340383727708172
Compute iterate 230 with error 8.911308481529545e-5
Compute iterate 240 with error 5.924528747414115e-5
Compute iterate 250 with error 3.9388200878676116e-5
Compute iterate 260 with error 2.6186561168373146e-5
Compute iterate 270 with error 1.7409680953761608e-5
Compute iterate 280 with error 1.1574523867352582e-5
Converged in 284 steps
```


In [ ]:
fig, ax = subplots(figsize=(9, 5))
ax[:set_ylim](-35, -24)
ax[:plot](grid_y, v_star_approx, lw=2, alpha=0.6, label="approximate value function")
ax[:plot](grid_y, v_star.(grid_y), lw=2, alpha=0.6, label="true value function")
ax[:legend](loc="lower right")
show()

In [ ]:
Tw, σ = bellman_operator(v_star_approx,
                         grid_y,
                         β,
                         log,
                         k -> k^α,
                         shocks;
                         compute_policy=true)


cstar = (1 - α * β) * grid_y

fig, ax = subplots(figsize=(9, 5))
ax[:plot](grid_y, σ, lw=2, alpha=0.6, label="approximate policy function")
ax[:plot](grid_y, cstar, lw=2, alpha=0.6, label="true policy function")
ax[:legend](loc="lower right")
show()

In [ ]:
s = 0.05
shocks = exp.(μ + s * randn(shock_size))

In [ ]:
"""
Compute a time series given consumption policy σ.
"""
function simulate_og(σ, y0 = 0.1, ts_length=100)
    y = Array{Float64}(ts_length)
    ξ = randn(ts_length-1)
    y[1] = y0
    for t in 1:(ts_length-1)
        y[t+1] = (y[t] - σ(y[t]))^α * exp(μ + s * ξ[t])
    end
    return y
end

fig, ax = subplots(figsize=(9, 6))

for β in (0.9, 0.94, 0.98)

    Tw = similar(grid_y)
    initial_w = 5 * log.(grid_y)

    v_star_approx = compute_fixed_point(bellman_operator,
                                        initial_w,
                                        max_iter=50,
                                        verbose=0,
                                        print_skip=10,
                                        err_tol=1e-5)

    Tw, σ = bellman_operator(v_star_approx,
                             grid_y,
                             β,
                             log,
                             k -> k^α,
                             shocks,
                             compute_policy=true)

    σ_func = LinInterp(grid_y, σ)
    y = simulate_og(σ_func)
    ax[:plot](y, lw=2, alpha=0.6, label="β = $β" )
end


ax[:legend](loc="lower right")
show()